This notebook implements the simulated studies for finite population estimand $\tau_{\rm SATE}$ based on ReR and QReR.

In [1]:
# load required package
import os
import itertools

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd

from tqdm import tqdm
from scipy.stats import ks_2samp

from datagen import *
from network import *

from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler

In [2]:
# generate parameter grid
def expand_grid(data_dict):
    """Create a dataframe from every combination of given values."""
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

net_params = {'data_path': ['./save/simu_data/'],
 'batch_size': [512],
 'lr': [0.001],
 'pa': [0.1,0.5,1],
 'val_metric': ['KS'],
 'num_nodes': [512],
 'num_iters': [5000],
 'num_init_iters': [500],
 'x_lambda': [1],
 'wt_lambda': [1],
 'patience': [15],
 'kernel_params': [{'kernel':'rbf',
                 'gamma':10,
                 'degree':2,
                 'c':0}],
 'random_state': [0]}

param_df = expand_grid(net_params)
param_df

,data_path,batch_size,lr,pa,val_metric,num_nodes,num_iters,num_init_iters,x_lambda,wt_lambda,patience,kernel_params,random_state
0,./save/simu_data/,512,0.001,0.1,KS,512,5000,500,1,1,15,"{'kernel': 'rbf', 'gamma': 10, 'degree': 2, 'c...",0
1,./save/simu_data/,512,0.001,0.5,KS,512,5000,500,1,1,15,"{'kernel': 'rbf', 'gamma': 10, 'degree': 2, 'c...",0
2,./save/simu_data/,512,0.001,1.0,KS,512,5000,500,1,1,15,"{'kernel': 'rbf', 'gamma': 10, 'degree': 2, 'c...",0


# Perform Simulation

In [3]:
# the folder directory to save the files
save_folder_root = './save/0415qrer_rer_sate/'

In [4]:
# function that generates response for ReR
def y_rer_gen(y,z,z_rer,tau=1):
    return y + tau*(z_rer - z)

In [5]:
def parallel_unit(i,r,scenario,batch_size,
                  lr,pa,
                  num_nodes,num_iters,
                  num_init_iters,
                  x_lambda,
                  wt_lambda,
                  kernel_params,val_metric,
                  patience,random_state,
                  data_path):
  

    ######## Load generated datasets ########
    # print('------------- Data:',i,'------------- ')

    # load the data
    data_full_path = data_path+ scenario+'/d' + str(i) + '.npy'
    dat = np.load(data_full_path,allow_pickle=True)

    # unzip the data
    x = dat.item()['x']
    z = dat.item()['z']
    y1 = dat.item()['y1']
    y2 = dat.item()['y2']
    y3 = dat.item()['y3']
    
    nt = int(z.sum())
    x = x[:nt*(1+r)]
    z = z[:nt*(1+r)]
    y1 = y1[:nt*(1+r)]
    y2 = y2[:nt*(1+r)]
    y3 = y3[:nt*(1+r)]
    
    sc = StandardScaler()
    xx = x.copy() # original x
    x = sc.fit_transform(x) # standardize the covariates
    
    
    ######## Perform QReR & ReR ########
    save_folder = save_folder_root+scenario+'/'+'r='+str(r)+'/pa='+str(pa)+'/'+str(i)+'/'
    
    if not os.path.exists(save_folder):
        # print('Creat the folder.')
        os.makedirs(save_folder)

    if not os.path.exists(save_folder+'final_checkpoint.pt'):
        # print('Train the model from scratch.')

        estimator = QReR(lr=lr,
                          batch_size=batch_size,
                          patience=patience,
                          num_iters=num_iters,
                          num_init_iters=num_init_iters,
                          pa=pa,
                          x_lambda=x_lambda,
                          wt_lambda=wt_lambda,
                          num_nodes=num_nodes,
                          val_metric=val_metric,
                          save_folder=save_folder,
                          kernel_params=kernel_params,
                          verbose=False,
                          random_state=random_state)

        # train the model from scratch
        estimator.fit(x,z)

    else:
        # load the model from saved file
        # print('Skip! The model has been trained.')
        estimator = QReR(lr=lr,
                          batch_size=batch_size,
                          patience=patience,
                          num_iters=num_iters,
                          num_init_iters=1,
                          pa=pa,
                          num_nodes=num_nodes,
                          val_metric=val_metric,
                          save_folder=save_folder,
                          kernel_params=kernel_params,
                          verbose=False,
                          random_state=random_state)
        estimator.w = z
        estimator.nwts = int(estimator.w.shape[0])
        estimator.nt = int(z.sum())
        estimator.nc = int((1-z).sum())
        estimator._init_network()
        estimator.netG.load_state_dict(torch.load(save_folder+'final_checkpoint.pt'))
    
    np.random.seed(i)
    torch.manual_seed(i)
    
    # generate allocation & weights
    z_rer = ReR(pa,torch.Tensor(x),np.sum(z))[0].numpy()
    wts_mat_net = estimator.predict(1000).numpy()
    
    # 1000 acceptable ReR allocations
    if not os.path.exists(save_folder+'zmat.npy'):
        z_rer_mat = np.array([ReR(pa,torch.Tensor(x),np.sum(z))[0].numpy() for i in range(1000)])
        np.save(save_folder+'zmat.npy',z_rer_mat)
    else:
        z_rer_mat = np.load(save_folder+'zmat.npy')
        
    # compute KS statistics and corresponding p-value to compare two methods
    mdiff_mat_net = np.array([cov_mdiff(x,z,wts_mat_net[i]) for i in range(1000)])
    mdiff_mat_rer = np.array([cov_mdiff(x,z_rer_mat[i]) for i in range(1000)])
    xmdiff_ks, xmdiff_pval = np.array([ks_2samp(mdiff_mat_net[:,i],
                                                mdiff_mat_rer[:,i]) for i in range(mdiff_mat_net.shape[1])]).mean(axis=0)
    
    if not os.path.exists(save_folder+'qrer_est.csv'):
        # Compare different strategies
        
        # QReR-S
        wts = wts_mat_net[0]

        est1_s = tau_diff(y1,z,wts)
        est2_s = tau_diff(y2,z,wts)
        est3_s = tau_diff(y3,z,wts)

        df_est_s = pd.DataFrame({
          'tauhat': [est1_s,est2_s,est3_s],
          'type': 'S'
        })
        
        # QReR-M
        avg_wts = wts_mat_net.mean(axis=0)

        est1_m = tau_diff(y1,z,avg_wts)
        est2_m = tau_diff(y2,z,avg_wts)
        est3_m = tau_diff(y3,z,avg_wts)

        df_est_m = pd.DataFrame({
          'tauhat': [est1_m,est2_m,est3_m],
          'type': 'M'
        })
        
        df_est = pd.concat([df_est_s,df_est_m],axis=0)
        df_est.to_csv(save_folder+"qrer_est.csv",index=False)
    else:
        pass
        # print('Skip! QReR has been considered')
        
    if not os.path.exists(save_folder+'rer_est.csv'):
    #if os.path.exists(save_folder+'rer_est.csv'):
        
        # generate data from the original covariates
        y1_rer, y2_rer, y3_rer = y_gen(xx,z_rer) 
        
        est1_rer = tau_diff(y1_rer,z_rer)
        est2_rer = tau_diff(y2_rer,z_rer)
        est3_rer = tau_diff(y3_rer,z_rer)
        
        df_est = pd.DataFrame({
          'tauhat': [est1_rer,est2_rer,est3_rer],
          'type': 'ReR'
        })
        
        df_est.to_csv(save_folder+"rer_est.csv",index=False)
    else:
        pass
        # print('Skip! ReR has been considered')

    return pd.read_csv(save_folder+"qrer_est.csv").values, pd.read_csv(save_folder+"rer_est.csv").values, xmdiff_ks, xmdiff_pval

In [6]:
def mcse_cal_fun(array_val,tau):
    # array value
    n_sim = array_val.shape[0]

    # bias MCSE
    bias_mcse = np.sqrt(np.var(array_val)/n_sim)

    # RMSE MCSE using asymptotic property
    # using Delta method and the formula in 
    # https://cran.r-project.org/web/packages/simhelpers/vignettes/MCSE.html
    array_sq_val = (array_val-tau)**2
    mse = array_sq_val.mean()
    rmse_mcse = np.sqrt(np.sum((array_sq_val-mse)**2)/((n_sim-1)*4*mse*n_sim))

    # # RMSE MCSE using bootstrapping
    # from sklearn.utils import resample
    # B = 1000 # bootstrapping iteration
    # rmse_boot_array = []
    # for b in range(B):
    #     array_val_boot = resample(array_val,
    #                               n_samples=n_sim,
    #                               replace=1)
    #     rmse_boot_array.append(np.sqrt(((array_val_boot-tau)**2).mean()))
    # rmse_boot_array = np.array(rmse_boot_array)
    # rmse_mcse = np.sqrt(np.var(rmse_boot_array))

    return bias_mcse, rmse_mcse

In [7]:
n_kernel = 40
n_data = 200
tau = 1

# data frames for QReR
result1_df = []
result2_df = []
result3_df = []

# data frames for ReR
result1_rer_df = []
result2_rer_df = []
result3_rer_df = []

# covariate mean difference 
xmdiff_df_lst = []

In [8]:
for scenario in ['scenario1','scenario2','scenario3']:
    for r in [1,2]:
        print('---------------------',scenario,'r =',r,'---------------------')
        for i_param in range(param_df.shape[0]):
            kwargs = dict(param_df.iloc[i_param,:])
            print('----------------- [%d/%d] -----------------\n'%(i_param+1,param_df.shape[0]))
            results_all = Parallel(n_jobs=n_kernel,verbose=0)(delayed(parallel_unit)(i=i,r=r,scenario=scenario,**kwargs) for i in tqdm(range(n_data)))

            # QReR
            dat_array = np.array([results_all[i][0] for i in range(n_data)])
            bias = dat_array[:,:,0].mean(axis=0)-tau
            rmse = np.sqrt(np.mean((dat_array[:,:,0]-tau)**2,axis=0).astype(float))
            mcse = np.apply_along_axis(func1d=mcse_cal_fun,
                                        arr=dat_array[:,:,0],
                                        axis=0,tau=tau)
            bias_mcse = mcse[0,:]
            rmse_mcse = mcse[1,:]

            result_dict = {'bias':bias,
                           'bias_mcse':bias_mcse,
                           'rmse':rmse,
                           'rmse_mcse':rmse_mcse,
                           'pa':param_df.iloc[i_param,3],
                           'method':['QReR-'+dat_array[0,i,1] for i in range(dat_array.shape[1])],
                           'r':r,
                           'scenario':scenario}
            result_df = pd.DataFrame(result_dict)
            result1_df.append(result_df.iloc[[0,3],:])
            result2_df.append(result_df.iloc[[1,4],:])
            result3_df.append(result_df.iloc[[2,5],:])
            
            # ReR
            dat_array = np.array([results_all[i][1] for i in range(n_data)])
            bias = dat_array[:,:,0].mean(axis=0)-tau
            rmse = np.sqrt(np.mean((dat_array[:,:,0]-tau)**2,axis=0).astype(float))
            mcse = np.apply_along_axis(func1d=mcse_cal_fun,
                                        arr=dat_array[:,:,0],
                                        axis=0,tau=tau)
            bias_mcse = mcse[0,:]
            rmse_mcse = mcse[1,:]

            result_dict = {'bias':bias,
                           'bias_mcse':bias_mcse,
                           'rmse':rmse,
                           'rmse_mcse':rmse_mcse,
                           'pa':param_df.iloc[i_param,3],
                           'method':'ReR',
                           'r':r,
                           'scenario':scenario}
            result_df = pd.DataFrame(result_dict)
            result1_rer_df.append(result_df.iloc[0,:])
            result2_rer_df.append(result_df.iloc[1,:])
            result3_rer_df.append(result_df.iloc[2,:])
            
            ## check the covariate mean difference
            ks_vals = np.array([results_all[i][2] for i in range(n_data)])
            ks_mean = np.mean(ks_vals)
            ks_mcse = np.sqrt(np.var(ks_vals)/ks_vals.shape[0])
            
            pval_vals = np.array([results_all[i][3] for i in range(n_data)])
            pval_mean = np.mean(pval_vals)
            pval_mcse = np.sqrt(np.var(pval_vals)/pval_vals.shape[0])
            
            xmdiff_dict = {'r':r,
                           'scenario':scenario,
                           'pa':param_df.iloc[i_param,3],
                           'KS':ks_mean,
                           'KS_MCSE':ks_mcse,
                           'p-val':pval_mean,
                           'p-val_MCSE':pval_mcse}
            xmdiff_df = pd.DataFrame(xmdiff_dict,index=[0])
            xmdiff_df_lst.append(xmdiff_df)

--------------------- scenario1 r = 1 ---------------------
----------------- [1/3] -----------------



 20%|████████▏                                | 40/200 [00:00<00:01, 102.27it/s]

Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.


  0%|          | 0/1 [00:00<?, ?it/s]

Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Initialize the network via pretraining.
Initialize the network via pretraining.


...

100%|██████████| 1/1 [00:00<00:00, 18.02it/s]


Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretrain

100%|██████████| 1/1 [00:00<00:00, 15.01it/s]


Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretraining complete!
Initialize the network via pretraining.
Pretrain

100%|██████████| 1/1 [00:00<00:00, 16.10it/s]


# Generate Tables

In [9]:
outcome1 = [pd.concat([result1_df[i],pd.DataFrame(result1_rer_df[i]).T],axis=0) for i,_ in enumerate(result1_df)]
outcome1_df = pd.concat(outcome1,axis=0)
outcome1_df['outcome'] = 'Linear'
outcome1_df.set_index(['scenario','r','method'])

bias bias_mcse      rmse rmse_mcse   pa outcome
scenario  r method                                                     
scenario1 1 QReR-S -0.001776  0.024628  0.348304  0.016894  0.1  Linear
            QReR-M -0.021587  0.008218  0.118204  0.005343  0.1  Linear
            ReR     0.004139  0.021849   0.30902  0.013167  0.1  Linear
            QReR-S -0.053232  0.033215  0.472742  0.025907  0.5  Linear
            QReR-M -0.036631  0.007975  0.118577  0.005475  0.5  Linear
            ReR     0.029207  0.031543  0.447042  0.021127  0.5  Linear
            QReR-S -0.035239  0.042031  0.595453  0.032167  1.0  Linear
            QReR-M -0.053685  0.007893  0.123858  0.005876  1.0  Linear
            ReR    -0.023117  0.040609  0.574763  0.027256  1.0  Linear
          2 QReR-S -0.032241  0.021455  0.305128  0.017697  0.1  Linear
            QReR-M -0.017174  0.007127  0.102246  0.005223  0.1  Linear
            ReR    -0.000725  0.019388  0.274191  0.013195  0.1  Linear
            QReR-S -0.018097  0.028472  0.403068  0.018523  0.5  Linear
            QReR-M  -0.03416   0.00711    0.1062  0.005444  0.5  Linear
            ReR     0.025591  0.026136  0.370507  0.018432  0.5  Linear
            QReR-S -0.030974  0.034412  0.487642  0.026693  1.0  Linear
            QReR-M -0.036516  0.006911   0.10434  0.005011  1.0  Linear
            ReR    -0.021953  0.033698  0.477069  0.021627  1.0  Linear
scenario2 1 QReR-S  0.004715  0.031089  0.439692  0.026101  0.1  Linear
            QReR-M -0.031854  0.007913  0.116345   0.00561  0.1  Linear
            ReR     0.001218   0.02723  0.385086  0.017667  0.1  Linear
            QReR-S -0.058799  0.042576  0.604977  0.032055  0.5  Linear
            QReR-M -0.059126  0.008228  0.130519  0.006567  0.5  Linear
            ReR     0.037899  0.037056  0.525422  0.024208  0.5  Linear
            QReR-S -0.016669  0.047707  0.674883  0.034048  1.0  Linear
            QReR-M -0.064157  0.008717  0.138974  0.007134  1.0  Linear
            ReR    -0.037723  0.048093  0.681188  0.033238  1.0  Linear
          2 QReR-S -0.050985  0.025319   0.36168  0.020362  0.1  Linear
            QReR-M -0.017676  0.007028  0.100944  0.004892  0.1  Linear
            ReR     0.018958  0.020799  0.294759  0.015114  0.1  Linear
            QReR-S -0.016125  0.034981  0.494966  0.023018  0.5  Linear
            QReR-M  -0.02922  0.006975  0.102873  0.004967  0.5  Linear
            ReR      0.04425  0.029559  0.420359  0.020863  0.5  Linear
            QReR-S -0.053056  0.038499  0.547032  0.030884  1.0  Linear
            QReR-M  -0.04407   0.00712  0.109909   0.00541  1.0  Linear
            ReR     -0.01471  0.038548  0.545343  0.023869  1.0  Linear
scenario3 1 QReR-S  0.020248  0.029421  0.416569  0.024012  0.1  Linear
            QReR-M -0.028013  0.007932  0.115622  0.005578  0.1  Linear
            ReR       0.0034  0.029298  0.414354  0.017737  0.1  Linear
            QReR-S -0.066989  0.046265  0.657705  0.041001  0.5  Linear
            QReR-M -0.057775  0.008358  0.131561  0.006099  0.5  Linear
            ReR    -0.001433  0.039612  0.560199   0.02389  0.5  Linear
            QReR-S -0.062409  0.053529  0.759587  0.034309  1.0  Linear
            QReR-M -0.060028  0.008816  0.138378  0.006953  1.0  Linear
            ReR     0.071693  0.050988  0.724637  0.038189  1.0  Linear
          2 QReR-S  0.011218  0.025053  0.354486  0.021037  0.1  Linear
            QReR-M -0.013105  0.006806   0.09714  0.005332  0.1  Linear
            ReR     0.031025  0.022773  0.323549  0.015315  0.1  Linear
            QReR-S -0.069196    0.0354  0.505388  0.024942  0.5  Linear
            QReR-M -0.036433  0.007292  0.109372  0.005808  0.5  Linear
            ReR     -0.01962  0.030731   0.43504  0.019822  0.5  Linear
            QReR-S  0.011723  0.037757  0.534093  0.024748  1.0  Linear
            QReR-M  -0.04697  0.007058  0.110319  0.006075  1.0  Linear
            ReR    -0.066887  0.040077  0.570704  0.028444  1.0 

In [10]:
outcome2 = [pd.concat([result2_df[i],pd.DataFrame(result2_rer_df[i]).T],axis=0) for i,_ in enumerate(result2_df)]
outcome2_df = pd.concat(outcome2,axis=0)
outcome2_df['outcome'] = 'NonLinear1'
outcome2_df.set_index(['scenario','r','method'])

bias bias_mcse      rmse rmse_mcse   pa     outcome
scenario  r method                                                         
scenario1 1 QReR-S -0.017444  0.034883  0.493634  0.022725  0.1  NonLinear1
            QReR-M -0.032721  0.013433  0.192763  0.009095  0.1  NonLinear1
            ReR     0.013678   0.03286  0.464913  0.019582  0.1  NonLinear1
            QReR-S -0.078672  0.045969  0.654843  0.037323  0.5  NonLinear1
            QReR-M -0.054856  0.012778  0.188849  0.009234  0.5  NonLinear1
            ReR     0.040662  0.044154  0.625759  0.028087  0.5  NonLinear1
            QReR-S -0.060728  0.059425  0.842593  0.045915  1.0  NonLinear1
            QReR-M -0.074569  0.012951  0.197752  0.010221  1.0  NonLinear1
            ReR    -0.037137    0.0565  0.799893  0.042368  1.0  NonLinear1
          2 QReR-S -0.102396  0.030282  0.440319  0.021605  0.1  NonLinear1
            QReR-M -0.080673  0.011734  0.184509   0.00894  0.1  NonLinear1
            ReR     0.003605  0.028043  0.396611  0.019387  0.1  NonLinear1
            QReR-S -0.088829  0.040049  0.573305  0.027881  0.5  NonLinear1
            QReR-M -0.096619  0.011892  0.193956  0.009407  0.5  NonLinear1
            ReR     0.038351  0.036889  0.523097  0.026286  0.5  NonLinear1
            QReR-S -0.086001  0.049074  0.699321  0.039141  1.0  NonLinear1
            QReR-M -0.092043  0.011958  0.192535  0.009139  1.0  NonLinear1
            ReR    -0.017879  0.046173  0.653234  0.028208  1.0  NonLinear1
scenario2 1 QReR-S -0.059761  0.044862  0.637252  0.034937  0.1  NonLinear1
            QReR-M  -0.12326   0.01489  0.243993  0.010771  0.1  NonLinear1
            ReR     0.011235  0.038442  0.543771  0.023851  0.1  NonLinear1
            QReR-S -0.162477  0.057748   0.83269  0.042144  0.5  NonLinear1
            QReR-M -0.148254  0.014667  0.254952  0.012009  0.5  NonLinear1
            ReR     0.052758  0.051919  0.736139  0.032804  0.5  NonLinear1
            QReR-S   -0.0955  0.065979  0.937957  0.050767  1.0  NonLinear1
            QReR-M -0.147682  0.014513   0.25286  0.013098  1.0  NonLinear1
            ReR    -0.058421  0.066774  0.946127  0.052491  1.0  NonLinear1
          2 QReR-S -0.207434  0.036765  0.559788  0.033137  0.1  NonLinear1
            QReR-M -0.142635  0.013205  0.234991  0.011001  0.1  NonLinear1
            ReR     0.029876  0.030046   0.42597  0.021936  0.1  NonLinear1
            QReR-S -0.122414  0.047661  0.685056  0.035729  0.5  NonLinear1
            QReR-M -0.148707   0.01316  0.238224  0.011721  0.5  NonLinear1
            ReR      0.05579  0.041536  0.590056  0.030444  0.5  NonLinear1
            QReR-S -0.167309  0.055159  0.797813  0.049608  1.0  NonLinear1
            QReR-M -0.156834  0.013771  0.250052  0.010852  1.0  NonLinear1
            ReR     -0.00881  0.052518  0.742768  0.031477  1.0  NonLinear1
scenario3 1 QReR-S -0.071345  0.043267  0.616031  0.036155  0.1  NonLinear1
            QReR-M -0.132334  0.015365  0.254417    0.0114  0.1  NonLinear1
            ReR     0.032095  0.043877  0.621342  0.025157  0.1  NonLinear1
            QReR-S -0.198502  0.064358  0.931561  0.059704  0.5  NonLinear1
            QReR-M -0.165342  0.015474   0.27428  0.011308  0.5  NonLinear1
            ReR    -0.020286     0.054  0.763944  0.031675  0.5  NonLinear1
            QReR-S -0.166057  0.077095  1.102858  0.052907  1.0  NonLinear1
            QReR-M -0.151813  0.015751  0.269568  0.012097  1.0  NonLinear1
            ReR     0.090318  0.071689  1.017849  0.053965  1.0  NonLinear1
          2 QReR-S  -0.12066  0.035103   0.51089  0.028701  0.1  NonLinear1
            QReR-M -0.145517  0.014198  0.247973  0.012181  0.1  NonLinear1
            ReR     0.041972  0.032235  0.457801  0.020849  0.1  NonLinear1
            QReR-S -0.201036  0.051852  0.760361  0.037258  0.5  NonLinear1
            QReR-M -0.166649  0.013694  0.255498   0.01127  0.5  NonLinear1
            ReR    -0.030289  0.043304  0.613165  0.027658  0.5  NonLine

In [11]:
outcome3 = [pd.concat([result3_df[i],pd.DataFrame(result3_rer_df[i]).T],axis=0) for i,_ in enumerate(result3_df)]
outcome3_df = pd.concat(outcome3,axis=0)
outcome3_df['outcome'] = 'NonLinear2'
outcome3_df.set_index(['scenario','r','method'])

bias bias_mcse      rmse rmse_mcse   pa     outcome
scenario  r method                                                         
scenario1 1 QReR-S  0.078184  0.163497  2.313521  0.134576  0.1  NonLinear2
            QReR-M  0.025263  0.144043  2.037234  0.126093  0.1  NonLinear2
            ReR     0.030233  0.143299  2.026781   0.09175  0.1  NonLinear2
            QReR-S  -0.05149  0.179258   2.53561  0.189097  0.5  NonLinear2
            QReR-M -0.062684  0.149793  2.119314  0.135547  0.5  NonLinear2
            ReR    -0.007234  0.137366  1.942658  0.084704  0.5  NonLinear2
            QReR-S -0.050613  0.187287  2.649125  0.236781  1.0  NonLinear2
            QReR-M -0.070518   0.15166  2.145959  0.135764  1.0  NonLinear2
            ReR    -0.048441  0.146449  2.071674  0.083338  1.0  NonLinear2
          2 QReR-S -0.281432  0.122546  1.755767   0.11978  0.1  NonLinear2
            QReR-M -0.266885  0.113389  1.625625  0.124947  0.1  NonLinear2
            ReR     0.100197  0.120083  1.701177  0.102444  0.1  NonLinear2
            QReR-S -0.298005  0.134963  1.931784  0.162191  0.5  NonLinear2
            QReR-M  -0.30778  0.119223  1.713936  0.153005  0.5  NonLinear2
            ReR     0.113986  0.117645  1.667658  0.097816  0.5  NonLinear2
            QReR-S -0.294104  0.137807  1.970956  0.117487  1.0  NonLinear2
            QReR-M -0.309935  0.121294  1.743134  0.134195  1.0  NonLinear2
            ReR     0.029843  0.122781  1.736645  0.096212  1.0  NonLinear2
scenario2 1 QReR-S  5.001272   0.39125  7.458421  0.498685  0.1  NonLinear2
            QReR-M  4.869004  0.357452  7.018656  0.442962  0.1  NonLinear2
            ReR    -0.571033  0.404642  5.750929  0.316897  0.1  NonLinear2
            QReR-S  4.958095  0.421366  7.751945  0.569212  0.5  NonLinear2
            QReR-M   4.86121  0.378251  7.228147   0.50597  0.5  NonLinear2
            ReR    -0.288543  0.395668   5.60303   0.29887  0.5  NonLinear2
            QReR-S  4.774445  0.412663  7.540128  0.549488  1.0  NonLinear2
            QReR-M  4.702679  0.371645  7.052604  0.481376  1.0  NonLinear2
            ReR    -0.028993  0.426295  6.028792  0.306556  1.0  NonLinear2
          2 QReR-S  4.671454  0.384901  7.173023  0.576421  0.1  NonLinear2
            QReR-M  4.819175  0.340768  6.815348  0.531337  0.1  NonLinear2
            ReR     0.093826  0.302443  4.278219  0.379452  0.1  NonLinear2
            QReR-S  5.054612  0.370016    7.2754  0.472076  0.5  NonLinear2
            QReR-M   4.84031  0.349117  6.914125   0.48851  0.5  NonLinear2
            ReR      0.23765  0.299368  4.240372  0.319625  0.5  NonLinear2
            QReR-S  4.871727   0.38528  7.309022  0.423826  1.0  NonLinear2
            QReR-M  4.858597  0.360734  7.044985  0.530661  1.0  NonLinear2
            ReR     0.325369  0.303956  4.310881  0.258657  1.0  NonLinear2
scenario3 1 QReR-S -5.931119  0.345938  7.688485  0.379078  0.1  NonLinear2
            QReR-M -6.118033   0.31615  7.577633  0.324788  0.1  NonLinear2
            ReR     0.367722  0.406989  5.767428  0.284704  0.1  NonLinear2
            QReR-S -6.869215  0.385166  8.766791  0.394901  0.5  NonLinear2
            QReR-M -6.551291  0.321868  7.977416    0.3291  0.5  NonLinear2
            ReR     0.119396  0.435633   6.16194  0.355136  0.5  NonLinear2
            QReR-S -7.007799  0.372343  8.765675  0.426292  1.0  NonLinear2
            QReR-M -7.229036  0.331125  8.613229  0.335946  1.0  NonLinear2
            ReR    -0.019949  0.458072  6.478152   0.38107  1.0  NonLinear2
          2 QReR-S -4.673931  0.321523   6.52082  0.407975  0.1  NonLinear2
            QReR-M -4.817004  0.281041  6.245024  0.305869  0.1  NonLinear2
            ReR    -0.322891  0.324746  4.603944  0.319908  0.1  NonLinear2
            QReR-S -5.632355  0.331105  7.324579   0.34048  0.5  NonLinear2
            QReR-M -5.519346  0.290509   6.88057  0.292525  0.5  NonLinear2
            ReR    -0.381521   0.35677    5.0599  0.329462  0.5  NonLine

In [12]:
outcome_df = pd.concat([outcome1_df,outcome2_df,outcome3_df])
outcome_df = outcome_df.set_index(['r','scenario','outcome','pa','method'])
outcome_df

bias bias_mcse      rmse rmse_mcse
r scenario  outcome    pa  method                                        
1 scenario1 Linear     0.1 QReR-S -0.001776  0.024628  0.348304  0.016894
                           QReR-M -0.021587  0.008218  0.118204  0.005343
                           ReR     0.004139  0.021849   0.30902  0.013167
                       0.5 QReR-S -0.053232  0.033215  0.472742  0.025907
                           QReR-M -0.036631  0.007975  0.118577  0.005475
...                                     ...       ...       ...       ...
2 scenario3 NonLinear2 0.5 QReR-M -5.519346  0.290509   6.88057  0.292525
                           ReR    -0.381521   0.35677    5.0599  0.329462
                       1.0 QReR-S -5.436983  0.346844  7.322634  0.349353
                           QReR-M -5.925297  0.303716  7.318321   0.31129
                           ReR     0.011615   0.34687  4.905493  0.231743

[162 rows x 4 columns]

In [13]:
outcome_df.to_csv(save_folder_root+'qrer_rer_sate.csv')

In [14]:
xmdiff_df = pd.concat(xmdiff_df_lst)
xmdiff_df

,r,scenario,pa,KS,KS_MCSE,p-val,p-val_MCSE
0,1,scenario1,0.1,0.083474,0.000546,0.123718,0.004530
0,1,scenario1,0.5,0.072831,0.000560,0.149606,0.004933
0,1,scenario1,1.0,0.067961,0.000557,0.160749,0.005236
0,2,scenario1,0.1,0.081227,0.000581,0.129380,0.004833
0,2,scenario1,0.5,0.071459,0.000563,0.148111,0.004941
0,2,scenario1,1.0,0.067149,0.000554,0.157053,0.005689
0,1,scenario2,0.1,0.083603,0.000602,0.132868,0.004956
0,1,scenario2,0.5,0.075149,0.000578,0.135246,0.004692
0,1,scenario2,1.0,0.068587,0.000563,0.150841,0.005056
0,2,scenario2,0.1,0.079976,0.000557,0.133432,0.004769


In [16]:
xmdiff_df.to_csv(save_folder_root+'xmdiff_qrer_rer_sate.csv')